In [246]:

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#%%

#in_fol = '/Volumes/New Volume/Other_works/tropcyc/atlantic_exps2/preprocessing/'
#in_fol = '/home/nmathewa/main/GIT/tropcyc/atlantic_exps2/preprocessing/'
in_fol = '/Users/nalex2023/main/tropcyc/atlantic_exps3/Preprocessing/'


#cifar_data = tf.keras.datasets.cifar10.load_data()

#y_labels = cifar_data[0][1]

#x = cifar_data[0][0]



x_data = np.load(in_fol+'final_arrv3.npy')

y_speeds = pd.read_csv(in_fol+'targetsv3.csv')['USA_WIND'].to_numpy()

#plt.plot(y_speeds)

#%%

support_file = pd.read_csv(in_fol+'support_file3.csv').set_index('id')

support_file['seq_no'] = support_file.groupby('id').count()['lead_time']

sequences = pd.unique(support_file['seq_no'])




#%%


x_train, x_test, y_train, y_test = train_test_split(x_data, y_speeds, test_size=0.2, random_state=1)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

#train_y_new = scaler.fit_transform(y_train)


import matplotlib.pyplot as plt


max_vals = []
min_vals = []


for ii in range(x_train.shape[-1]):
    max_vals += [x_train[:,:,:,ii].max()]
    min_vals += [x_train[:,:,:,ii].min()]




norm_x_train = (x_train - min_vals)/(np.array(max_vals) - np.array(min_vals))

#norm_y_train = (y_train - y_train.mean())/y_train.std()

norm_y_train = (y_train - y_train.min())/(y_train.max() - y_train.min())

max_vals = []
min_vals = []

for jj in range(x_test.shape[-1]):
    max_vals += [x_test[:,:,:,jj].max()]
    min_vals += [x_test[:,:,:,jj].min()]


norm_x_test = (x_test - min_vals)/(np.array(max_vals) - np.array(min_vals))

#norm_y_train = (y_train - y_train.mean())/y_train.std()

norm_y_test = (y_test - y_test.min())/(y_test.max() - y_test.min())



#%% Sequencing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.regularizers import l1_l2
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,LSTM, TimeDistributed, ConvLSTM2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import ZeroPadding3D
from tensorflow.keras.layers import Reshape



max_len = sequences.max()


# convert tjhe 3d data in to sequences


padded_seq_x = []
padded_seq_y = [] 

for ii in sequences:
    frame1 = norm_x_train[:ii,:,:,:]
    padded_seq_x += [np.pad(frame1,((0,max_len-ii),(0,0),(0,0),(0,0)))]

for ii in sequences:
    frame1 = norm_y_train[:ii]
    padded_seq_y += [np.pad(frame1,((0,max_len-ii)))]


model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(max_len,40,40,9)))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(128, return_sequences=True,input_shape=(max_len,32,32,9)))

model.add(TimeDistributed(Dense(1, activation='relu')))



model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])






In [248]:
model.fit(np.array(padded_seq), np.array(padded_seq_y), epochs=10, batch_size=32, verbose=1)

Epoch 1/10
2/2 [==============================] - 3s 1s/step - loss: 0.0347 - mean_squared_error: 0.0347
Epoch 2/10
2/2 [==============================] - 3s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 3/10
2/2 [==============================] - 2s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 4/10
2/2 [==============================] - 3s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 5/10
2/2 [==============================] - 2s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 6/10
2/2 [==============================] - 3s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 7/10
2/2 [==============================] - 2s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 8/10
2/2 [==============================] - 2s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 9/10
2/2 [==============================] - 2s 1s/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 10/10
2/2 [==============================] - 2s 1

Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [224]:
padded_seq[0].shape

(12, 71)

In [151]:
data3d.shape

(4608, 40, 360)

In [174]:
model.fit(padded_seq, norm_y_train, epochs=10, batch_size=1, verbose=1,)

Epoch 1/10


ValueError: in user code:

    File "/Users/nalex2023/miniconda3/envs/deep/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/nalex2023/miniconda3/envs/deep/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/nalex2023/miniconda3/envs/deep/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/nalex2023/miniconda3/envs/deep/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/nalex2023/miniconda3/envs/deep/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/nalex2023/minico

In [100]:
sequences

array([12, 30, 13,  9, 20, 10,  7, 14, 28, 32, 17,  5,  4, 15, 26, 27,  3,
       18,  2, 33,  1, 31, 49, 43, 19,  8, 35, 24,  6, 71, 51, 56, 54, 40,
       25, 36, 21, 11, 23, 29, 37, 39, 42, 41, 48, 38, 46, 44, 60, 47, 16,
       45, 53, 22])

In [72]:


def train_generator(sequences):
    while True:
        index = 0

        for ii in range(len(sequences)):
            seq_len = sequences[index % len(sequences)]
            end_index = index + seq_len
            end_index = min(end_index, len(x_train))
            x = norm_x_train[index:end_index,:,:,:]
            y = norm_y_train[index:end_index]
            yield x, y
            index += seq_len

(12, 40, 40, 9)
(4, 40, 40, 9)
(3, 40, 40, 9)
(33, 40, 40, 9)
(53, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)
(42, 40, 40, 9)
(51, 40, 40, 9)
(24, 40, 40, 9)
(45, 40, 40, 9)


In [46]:
x.shape

(12, 40, 40, 9)